#Libarary

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np

# Load Data

In [ ]:
# Google Drive mounting code removed for GitHub

Mounted at /content/drive/


In [ ]:
# Load data from CSV files
rfm_df = pd.read_csv('path_to_rfm_data.csv', low_memory=False)  # replace with your file path
invoice_df = pd.read_csv('path_to_invoice_data.csv', low_memory=False)  # replace with your file path

In [ ]:
# Make sure 'Invoice_date' is in datetime format
invoice_df['Invoice_date'] = pd.to_datetime(invoice_df['Invoice_date'])

# Feature Engineering

## Customer Behavior Features

In [ ]:
# Define a reference date, e.g., the latest date in the dataset or a specific date
reference_date = invoice_df['Invoice_date'].max()  # Using max date in the dataset as reference

# Group data by each customer
customer_features = invoice_df.groupby('CompanyCustomerKey').apply(lambda x: pd.Series({
    # 1. Average Order Value (AOV): Total Gross Margin (GM_Dollars) / Total Quantity (Qty_Shipped)
    'avg_order_value': x['GM_Dollars'].sum() / x['Qty_Shipped'].sum() if x['Qty_Shipped'].sum() > 0 else 0,

    # 2. Order Count: Number of unique 'InvoiceNo' per customer
    'order_count': x['InvoiceNo'].nunique(),

    # 3. Average Days Between Orders
    'avg_days_between_orders': x['Invoice_date'].sort_values().diff().mean().days if len(x) > 1 else 0,

    # 4. Days Since Last Order: Difference between reference date and last order date
    'days_since_last_order': (reference_date - x['Invoice_date'].max()).days
})).reset_index()

# Display the resulting DataFrame with the new customer behavior features
customer_features


<ipython-input-8-07b8b69f8834>:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  customer_features = invoice_df.groupby('CompanyCustomerKey').apply(lambda x: pd.Series({


,CompanyCustomerKey,avg_order_value,order_count,avg_days_between_orders,days_since_last_order
0,9468030,276.852000,1.0,0.0,279.0
1,10582379,16.820458,33.0,95.0,30.0
2,11098756,9.322858,66.0,5.0,2.0
3,11133877,28.448065,7.0,103.0,571.0
4,11194873,42.770645,8.0,177.0,61.0
...,...,...,...,...,...
17156,51505077,1132.818000,1.0,0.0,1.0
17157,51505283,126.840000,1.0,0.0,2.0
17158,52099201,2349.260000,1.0,0.0,151.0
17159,52099203,7.007345,4.0,24.0,27.0


## Product and Purchase Features



*   Unique Product Count: Number of distinct products (CompanyProdKey) purchased by each customer.
*   Most Frequent Product Category: Most frequently purchased product category (NAICSCode) for each customer.

*   Product Variety Score: Ratio of unique products purchased to the total purchases.
*   High-value Product Count: Number of purchases where the gross margin (GM_Dollars) is above a specified threshold.











In [ ]:
# determine disterbution of GM_Dollars
invoice_df['GM_Dollars'].describe()

,GM_Dollars
count,5.277051e+06
mean,1.342904e+02
std,1.143729e+03
min,-5.362955e+05
25%,6.870000e+00
50%,2.490000e+01
75%,8.756000e+01
max,1.262488e+06


In [ ]:


# Define a threshold for high-value products (adjust based on your data distribution)
high_value_threshold = 25

# Calculate each feature for each customer
product_purchase_features = invoice_df.groupby('CompanyCustomerKey').apply(lambda x: pd.Series({
    # 1. Unique Product Count
    'unique_product_count': x['CompanyProdKey'].nunique(),

    # 2. Most Frequent Product Category (mode of NAICSCode)
    'most_frequent_product_NAICSCode': x['NAICSCode'].mode()[0] if not x['NAICSCode'].mode().empty else None,

    # 3. Product Variety Score: Unique Products / Total Purchases
    'product_variety_score': x['CompanyProdKey'].nunique() / x['InvoiceNo'].nunique() if x['InvoiceNo'].nunique() > 0 else 0,

    # 4. High-value Product Count: Number of orders where GM_Dollars > high_value_threshold
    'high_value_product_count': (x['GM_Dollars'] > high_value_threshold).sum()
})).reset_index()

# Display the resulting DataFrame with the new product and purchase features
product_purchase_features


<ipython-input-10-ce38790b2982>:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  product_purchase_features = invoice_df.groupby('CompanyCustomerKey').apply(lambda x: pd.Series({


,CompanyCustomerKey,unique_product_count,most_frequent_product_NAICSCode,product_variety_score,high_value_product_count
0,9468030,1.0,541380.0,1.000000,1.0
1,10582379,12.0,333120.0,0.363636,16.0
2,11098756,165.0,238910.0,2.500000,121.0
3,11133877,6.0,237120.0,0.857143,7.0
4,11194873,1.0,532490.0,0.125000,8.0
...,...,...,...,...,...
17156,51505077,1.0,999999.0,1.000000,1.0
17157,51505283,1.0,333611.0,1.000000,1.0
17158,52099201,1.0,327420.0,1.000000,1.0
17159,52099203,4.0,327420.0,1.000000,6.0


## Seasonality and Trend Features


*   Purchase Seasonality: Purchase trends based on seasons (e.g., whether the customer makes purchases mainly in certain months).
*  Purchase Day of Week and Month: Encode the day of the week and month when purchases are made, indicating possible purchasing habits.

*   Total Purchase Volume by Year: Trend of purchase volume or gross margin over years to identify loyal customers.











In [ ]:
# Extract month, day of the week, and year from the 'Invoice_date'
invoice_df['month'] = invoice_df['Invoice_date'].dt.month
invoice_df['day_of_week'] = invoice_df['Invoice_date'].dt.dayofweek  # 0 = Monday, 6 = Sunday
invoice_df['year'] = invoice_df['Invoice_date'].dt.year

# Calculate the seasonality and trend features for each customer
seasonality_trend_features = invoice_df.groupby('CompanyCustomerKey').apply(lambda x: pd.Series({
    # 1. Purchase Seasonality: Average purchase month (seasonality)
    'avg_purchase_month': x['month'].mode()[0] if not x['month'].mode().empty else None,

    # 2. Purchase Day of Week: Most common day of week for purchases
    'most_common_day_of_week': x['day_of_week'].mode()[0] if not x['day_of_week'].mode().empty else None,

    # 3. Total Purchase Volume by Year: Sum of GM_Dollars spent by customer per year
    'purchase_trend': x.groupby('year')['GM_Dollars'].sum().pct_change().mean()  # Average yearly percentage change
})).reset_index()

# Adding the 'total_purchase_volume_{year}' columns
for year in invoice_df['year'].unique():
    # Create a new column for total GM_Dollars spent by customer in each year
    seasonality_trend_features[f'total_purchase_volume_{year}'] = invoice_df.groupby('CompanyCustomerKey').apply(
        lambda x: x.loc[x['year'] == year, 'GM_Dollars'].sum()).reset_index(drop=True)

# Loop to create sum_gm_<year> columns
for year in seasonality_trend_features.columns:
    if 'total_purchase_volume_' in year:  # Only loop through columns that have yearly data
        year_value = int(year.split('_')[-1])  # Extract year from column name (e.g., 2023 from 'total_purchase_volume_2023')
        sum_gm_column_name = f'sum_gm_{year_value}'  # Name for the sum_gm column (e.g., 'sum_gm_2023')

        # Calculate sum_gm_<year> by summing the GM_Dollars for each year, or default to 0 if year is missing
        seasonality_trend_features[sum_gm_column_name] = seasonality_trend_features[year].apply(lambda x: x if pd.notnull(x) else 0)

# Display the resulting seasonality and trend features
seasonality_trend_features


<ipython-input-12-7ce36636bc14>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  seasonality_trend_features = invoice_df.groupby('CompanyCustomerKey').apply(lambda x: pd.Series({
<ipython-input-12-7ce36636bc14>:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  seasonality_trend_features[f'total_purchase_volume_{year}'] = invoice_df.groupby('CompanyCustomerKey').apply(
<ipython-input-12-7ce36636bc14>:21: Dep

,CompanyCustomerKey,avg_purchase_month,most_common_day_of_week,purchase_trend,total_purchase_volume_2023,total_purchase_volume_2019,total_purchase_volume_2020,total_purchase_volume_2022,total_purchase_volume_2017,total_purchase_volume_2018,...,total_purchase_volume_2024,sum_gm_2023,sum_gm_2019,sum_gm_2020,sum_gm_2022,sum_gm_2017,sum_gm_2018,sum_gm_2021,sum_gm_2016,sum_gm_2024
0,9468030,9.0,0.0,NaN,1384.26,0.00,0.00,0.00,0.00,0.00,...,0.00,1384.26,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,10582379,1.0,0.0,18.162788,471.95,47.37,158.62,3.68,458.53,125.55,...,444.00,471.95,47.37,158.62,3.68,458.53,125.55,0.00,493.78,444.00
2,11098756,12.0,1.0,0.194288,2506.70,0.00,2310.12,1037.22,0.00,0.00,...,2546.69,2506.70,0.00,2310.12,1037.22,0.00,0.00,1677.28,0.00,2546.69
3,11133877,12.0,2.0,0.393991,0.00,0.00,1944.48,1138.30,0.00,0.00,...,0.00,0.00,0.00,1944.48,1138.30,0.00,0.00,444.78,0.00,0.00
4,11194873,9.0,0.0,0.623018,195.92,0.00,206.03,630.06,0.00,0.00,...,293.88,195.92,0.00,206.03,630.06,0.00,0.00,0.00,0.00,293.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17156,51505077,6.0,5.0,NaN,0.00,0.00,0.00,0.00,0.00,0.00,...,5664.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5664.09
17157,51505283,6.0,4.0,NaN,0.00,0.00,0.00,0.00,0.00,0.00,...,126.84,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,126.84
17158,52099201,1.0,2.0,NaN,0.00,0.00,0.00,0.00,0.00,0.00,...,2349.26,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2349.26
17159,52099203,1.0,0.0,NaN,0.00,0.00,0.00,0.00,0.00,0.00,...,2823.96,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2823.96


## Early Purchase Behavior


*   Time to First Purchase: The time (in days) from the customer's first recorded purchase to the current date.
*   Total Spend in Early Purchases: The total gross margin (GM_Dollars) spent in the first few orders by the customer (you can define the "first few" as the first N orders).
*   Early Repeat Purchase Behavior: Whether the customer made a repeat purchase within a certain time frame after their first purchase (e.g., within 30 days).









In [ ]:


# Group by 'CompanyCustomerKey' and calculate early purchase behavior features
early_purchase_features = invoice_df.groupby('CompanyCustomerKey').apply(lambda x: pd.Series({
    # 1. Time to First Purchase: Time difference between the first purchase and the current date
    'time_to_first_purchase': (x['Invoice_date'].min() - x['Invoice_date'].max()).days,

    # 2. Total Spend in Early Purchases: Total GM_Dollars spent in the first N orders (define N)
    'total_spend_in_early_purchases': x.sort_values('Invoice_date').head(3)['GM_Dollars'].sum(),  # First 3 orders

    # 3. Early Repeat Purchase Behavior: Whether a repeat purchase occurred within 30 days of first purchase
    'early_repeat_purchase': (x['Invoice_date'].diff().dt.days <= 182).sum() > 0  # Check if any purchase occurred within 182 days
})).reset_index()

# Display the resulting DataFrame with the new early purchase behavior features
early_purchase_features


<ipython-input-13-772c2a989710>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  early_purchase_features = invoice_df.groupby('CompanyCustomerKey').apply(lambda x: pd.Series({


,CompanyCustomerKey,time_to_first_purchase,total_spend_in_early_purchases,early_repeat_purchase
0,9468030,0,1384.26,False
1,10582379,-3045,116.46,True
2,11098756,-1299,383.73,True
3,11133877,-722,1989.00,True
4,11194873,-1240,478.13,True
...,...,...,...,...
17156,51505077,0,5664.09,False
17157,51505283,0,126.84,False
17158,52099201,0,2349.26,False
17159,52099203,-124,1100.80,True


## Gross Margin-related Features


*   Total Gross Margin: The total gross margin (GM_Dollars) generated by each customer.




In [ ]:

# Calculate Gross Margin-related features for each customer
gross_margin_features = invoice_df.groupby('CompanyCustomerKey').apply(lambda x: pd.Series({
    # 1. Total Gross Margin: Total GM_Dollars for the customer
    'total_gross_margin': x['GM_Dollars'].sum()

})).reset_index()

# Display the resulting DataFrame with the new gross margin-related features
gross_margin_features


<ipython-input-14-dce80645eca8>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gross_margin_features = invoice_df.groupby('CompanyCustomerKey').apply(lambda x: pd.Series({


,CompanyCustomerKey,total_gross_margin
0,9468030,1384.26
1,10582379,2203.48
2,11098756,10078.01
3,11133877,3527.56
4,11194873,1325.89
...,...,...
17156,51505077,5664.09
17157,51505283,126.84
17158,52099201,2349.26
17159,52099203,2823.96


## Merge Features

In [ ]:
# Sequentially merge all dataframes on 'CompanyCustomerKey'

final_df = pd.merge(customer_features, product_purchase_features, on='CompanyCustomerKey', how='left')
final_df = pd.merge(final_df, seasonality_trend_features, on='CompanyCustomerKey', how='left')
final_df = pd.merge(final_df, early_purchase_features, on='CompanyCustomerKey', how='left')
final_df = pd.merge(final_df, gross_margin_features, on='CompanyCustomerKey', how='left')
final_df = pd.merge(final_df, rfm_df, on='CompanyCustomerKey', how='left')

# Display the final dataframe
final_df

,CompanyCustomerKey,avg_order_value,order_count,avg_days_between_orders,days_since_last_order,unique_product_count,most_frequent_product_NAICSCode,product_variety_score,high_value_product_count,avg_purchase_month,...,sum_gm_2016,sum_gm_2024,time_to_first_purchase,total_spend_in_early_purchases,early_repeat_purchase,total_gross_margin,frequency,recency,T,monetary_value
0,9468030,276.852000,1.0,0.0,279.0,1.0,541380.0,1.000000,1.0,9.0,...,0.00,0.00,0,1384.26,False,1384.26,0.0,0.0,279.0,0.000000
1,10582379,16.820458,33.0,95.0,30.0,12.0,333120.0,0.363636,16.0,1.0,...,493.78,444.00,-3045,116.46,True,2203.48,31.0,3045.0,3075.0,70.594194
2,11098756,9.322858,66.0,5.0,2.0,165.0,238910.0,2.500000,121.0,12.0,...,0.00,2546.69,-1299,383.73,True,10078.01,56.0,1299.0,1301.0,144.430179
3,11133877,28.448065,7.0,103.0,571.0,6.0,237120.0,0.857143,7.0,12.0,...,0.00,0.00,-722,1989.00,True,3527.56,6.0,722.0,1293.0,263.846667
4,11194873,42.770645,8.0,177.0,61.0,1.0,532490.0,0.125000,8.0,9.0,...,0.00,293.88,-1240,478.13,True,1325.89,6.0,1240.0,1301.0,186.643333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17156,51505077,1132.818000,1.0,0.0,1.0,1.0,999999.0,1.000000,1.0,6.0,...,0.00,5664.09,0,5664.09,False,5664.09,0.0,0.0,1.0,0.000000
17157,51505283,126.840000,1.0,0.0,2.0,1.0,333611.0,1.000000,1.0,6.0,...,0.00,126.84,0,126.84,False,126.84,0.0,0.0,2.0,0.000000
17158,52099201,2349.260000,1.0,0.0,151.0,1.0,327420.0,1.000000,1.0,1.0,...,0.00,2349.26,0,2349.26,False,2349.26,0.0,0.0,151.0,0.000000
17159,52099203,7.007345,4.0,24.0,27.0,4.0,327420.0,1.000000,6.0,1.0,...,0.00,2823.96,-124,1100.80,True,2823.96,3.0,124.0,151.0,691.153333


In [ ]:
# columns = ['CompanyCustomerKey','avg_order_value', 'order_count',
       'avg_days_between_orders', 'days_since_last_order', 'unique_product_count',
       'most_frequent_product_NAICSCode', 'product_variety_score',
       'high_value_product_count', 'avg_purchase_month',
       'most_common_day_of_week', 'purchase_trend','sum_gm_2023', 'sum_gm_2019',
       'sum_gm_2020', 'sum_gm_2022', 'sum_gm_2017', 'sum_gm_2018',
       'sum_gm_2021', 'sum_gm_2016', 'sum_gm_2024', 'time_to_first_purchase',
       'total_spend_in_early_purchases', 'early_repeat_purchase',
       'total_gross_margin','frequency','recency','T','monetary_value']

final_df = final_df[columns]

# Filter customer

##  Recency of Last Purchase

In [ ]:
# Filter customers who have made a purchase in the last 6 months
final_df_filtered = final_df[final_df['days_since_last_order'] <= 182]

# Filter customer with order_count>1
final_df_filtered = final_df[final_df['order_count']>1]

#Save

In [ ]:
# save
final_df_filtered = pd.to_csv('path_to_Feature_Engineering_CLV.csv')  # replace with your file path